# Analyse Time Performance

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import cProfile
from pstatsviewer import StatsViewer
import os
import holoviews as hv
hv.extension('bokeh')

In [ ]:
# Change to directory project 1
os.chdir("/Users/AndreCNF/Documents/AtariGoAI/project1")

In [ ]:
import go
from alphabeta_cutoff_search import alphabeta_cutoff_search

Name of the board to use:

In [ ]:
boardName = 'emptyBoard'

Script to test:

In [ ]:
mainScript = 'mainAiAi'

In [ ]:
statsFileName = mainScript + '_' + boardName + '_dictionaryGroups065035.stats'

### Functions with the main scripts that we want to test

It wont be able to fetch the code from the .py's in the directory for some reason.

In [ ]:
def mainAiAi():
    s = []
    endGame = 0

    myGame = go.Game()
    s = myGame.load_board(open('boards/' + boardName + '.txt', 'r'))

    while not endGame:
        # Print board
        # myGame.print_board(s)

        # AI 1's turn
        AiMove = alphabeta_cutoff_search(s, myGame, d=3)
        s = myGame.result(s, AiMove)
        endGame = myGame.terminal_test(s)

        # AI 2's turn
        AiMove = alphabeta_cutoff_search(s, myGame, d=3)
        s = myGame.result(s, AiMove)
        endGame = myGame.terminal_test(s)

    # Print the final board
    myGame.print_board(s, print_ids=True)

### cProfile Dataframe

Run cProfile on the main script of AI vs AI:

In [ ]:
cProfile.run(
    mainScript + '()',
    statsFileName,
)

In [ ]:
AiAiStats = StatsViewer(statsFileName)

Get a dataframe with the number of calls and running durations of each function:

In [ ]:
df = AiAiStats.timings
df.head()

### Sort data

Order by cumulative time:

In [ ]:
sortedData = df.sort_values(by='cumtime', ascending=False)
sortedData.head()

### Filter data

Filter data to only indicate functions that are relevant to us:

In [ ]:
relevantFunctions = ['go.py', 'alphabeta_cutoff_search.py', 'copy.py']

In [ ]:
['go.py' in idx for idx in sortedData.index]

In [ ]:
[any(func in idx for func in relevantFunctions) for idx in sortedData.index]

In [ ]:
# Temporary solution to filtering data without the Pandas apply() method
filteredData = sortedData[[any(func in idx for func in relevantFunctions) for idx in sortedData.index]]

### Organize data

In [ ]:
organizedData = filteredData.reset_index()

In [ ]:
organizedData[['filename', 'funcname']] = organizedData['filename:funcname'].str.split(":", expand=True)

In [ ]:
organizedData = organizedData.drop(columns='filename:funcname')

In [ ]:
organizedData.head()

In [ ]:
fileCumTimes = organizedData.groupby('filename').cumtime.sum()
fileCumTimes

### Plots

In [ ]:
%%opts Bars [xrotation=90 width=900 height=500 show_legend=False tools=['hover'] invert_axes=True ]
%%opts Bars (fill_color=Cycle('Spectral'))
%%opts Layout [shared_axes=False]

hv.Bars(filteredData, 'filename:funcname', 'cumtime')

In [ ]:
%%opts Bars [xrotation=90 width=900 height=500 show_legend=False tools=['hover'] invert_axes=True ]
%%opts Bars (fill_color=Cycle('Spectral'))
%%opts Layout [shared_axes=False]

hv.Bars(fileCumTimes, 'filename')

**Comment:** The copy methods take more time than all the rest of our code (from go.py) combined!

More options for the bar plot:

In [ ]:
hv.help(hv.Bars)